In [38]:
import numpy as np

chances = [0.25, 0.35, 0.45, 0.55, 0.65, 0.75]

def score(s_1, s_2, s_3):
    return s_1 + s_2 - s_3 - (abs(s_1 - s_2))


def make_lookup_table(facet_count):
    facet_count += 1
    dim = np.arange(facet_count)

    f = facet_count
    #                 chance, remainingx3, succsessesx3
    expected_facets = np.zeros([6,   f, f, f,     f, f, f], np.float16) - 500
    print("Gerenrating Table")
    for s_1 in dim:
        for s_2 in dim:
            for s_3 in dim:
                expected_facets[:,0 , 0, 0, s_1, s_2, s_3] = score(s_1, s_2, s_3)
    
    for r_1 in dim:
        print(r_1 / facet_count * 100, "%")
        for r_2 in dim:
            for r_3 in dim:
                if r_1 == r_2 == r_3 == 0:
                                continue
                for s_1 in range(facet_count - r_1)[::-1]:
                    for s_2 in range(facet_count - r_2)[::-1]:
                        for s_3 in range(facet_count - r_3)[::-1]:
                            for chance in np.arange(6):
                                suc_chance = chances[chance]
                                fail_chance = 1 - suc_chance
                                suc_chance_pointer = max(0, chance - 1)
                                fail_chance_pointer = min(5, chance + 1)

                                value = 0
                                if r_1 > 0:
                                    value = max(value, suc_chance * expected_facets[suc_chance_pointer, r_1 - 1, r_2, r_3, s_1 + 1, s_2, s_3] + fail_chance * expected_facets[fail_chance_pointer, r_1 - 1, r_2, r_3, s_1, s_2, s_3])
                                if r_2 > 0:
                                    value = max(value, suc_chance * expected_facets[suc_chance_pointer, r_1, r_2 - 1, r_3, s_1, s_2 + 1, s_3] + fail_chance * expected_facets[fail_chance_pointer, r_1, r_2 - 1, r_3, s_1, s_2, s_3])
                                if r_3 > 0:
                                    value = max(value, suc_chance * expected_facets[suc_chance_pointer, r_1, r_2, r_3 - 1, s_1, s_2, s_3 + 1] + fail_chance * expected_facets[fail_chance_pointer, r_1, r_2, r_3 - 1, s_1, s_2, s_3])

                                expected_facets[chance, r_1, r_2, r_3, s_1, s_2, s_3] = value




    return expected_facets

table = make_lookup_table(9)

Gerenrating Table
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [37]:
class Stone:
    def __init__(self, num_facets: int):
        self.num_facets = num_facets
        self.chance = 5
        self.row1 = []
        self.row2 = []
        self.row3 = []

    def copy(self):
        clone = Stone(self.num_facets)
        clone.chance = self.chance
        clone.row1 = self.row1.copy()
        clone.row2 = self.row2.copy()
        clone.row3 = self.row3.copy()
        return clone

    def to_array(self):
        return [self.chance, self.num_facets - len(self.row1), self.num_facets - len(self.row2), self.num_facets - len(self.row3), sum(self.row1), sum(self.row2), sum(self.row3)]

    def tap1(self, succ: bool):
        if len(self.row1) >= self.num_facets:
            return
        else:
            self.row1.append(succ)
        self.change_chance(succ)
    
    def tap2(self, succ: bool):
        if len(self.row2) >= self.num_facets:
            return
        else:
            self.row2.append(succ)
        self.change_chance(succ)
    
    def tap3(self, succ: bool):
        if len(self.row3) >= self.num_facets:
            return
        else:
            self.row3.append(succ)
        self.change_chance(succ)

    def change_chance(self, succ):
        if succ:
            self.chance = max(0, self.chance - 1)
        else:
            self.chance = min(5, self.chance + 1)

    def string(self):
        res = "  "
        for i in self.row1:
            res += " * " if i else " _ "

        res += "   "*(self.num_facets - len(self.row1))
        res += str(sum(self.row1)) + " Z U\n  "

        for i in self.row2:
            res += " * " if i else " _ "
        
        res += "   "*(self.num_facets - len(self.row2))
        res += str(sum(self.row2)) + " H J\n  "

        for i in self.row3:
            res += " * " if i else " _ "
        res += "   "*(self.num_facets - len(self.row3))
        res += str(sum(self.row3)) + " N M\n  "

        res += "\n   Backspace: Undo, Delete: Reset, Esc: Exit"
        return res

        
a = Stone(9)
a.tap1(1)
a.tap1(1)
a.tap1(1)
a.tap2(1)
a.tap2(1)
a.tap2(0)
a.tap2(0)
a.tap2(1)
a.tap3(0)
a.tap3(0)
a.tap3(0)
a.tap3(0)
a.tap3(0)
a.tap3(0)
a.tap3(0)

print(a.string())
print(a.to_array())


   *  *  *                   3 Z U
   *  *  _  _  *             3 H J
   _  _  _  _  _  _  _       0 N M
  
   Backspace: Undo, Delete: Reset, Esc: Exit
[5, 6, 4, 2, 3, 3, 0]


In [36]:
from pynput import keyboard
from IPython.display import clear_output

def get_score(stone, row):
    global table
    global chances
    global num_facets

    chance, r_1, r_2, r_3, s_1, s_2, s_3 = stone


    suc_chance = chances[chance]
    fail_chance = 1 - suc_chance
    suc_chance_pointer = max(0, chance - 1)
    fail_chance_pointer = min(5, chance + 1)

    if r_1 == 0 and row == 0:
        return -9
    if r_2 == 0 and row == 1:
        return -9
    if r_2 == 0 and row == 2:
        return -9

    if row == 0:
        return suc_chance * table[suc_chance_pointer, r_1 - 1, r_2, r_3, s_1 + 1, s_2, s_3] + fail_chance * table[fail_chance_pointer, r_1 - 1, r_2, r_3, s_1, s_2, s_3]
    if row == 1:
        return suc_chance * table[suc_chance_pointer, r_1, r_2 - 1, r_3, s_1, s_2 + 1, s_3] + fail_chance * table[fail_chance_pointer, r_1, r_2 - 1, r_3, s_1, s_2, s_3]
    if row == 2:
        return suc_chance * table[suc_chance_pointer, r_1, r_2, r_3 - 1, s_1, s_2, s_3 + 1] + fail_chance * table[fail_chance_pointer, r_1, r_2, r_3 - 1, s_1, s_2, s_3]
    
    

def on_press(key):
    global listener
    global stonestates
    global num_facets

    if key == keyboard.Key.esc:
        print("Exiting")
        listener.stop()
        return

    elif key == keyboard.Key.backspace:
        if len(stonestates) > 1:
            stonestates = stonestates[:-1]
        cur_stone = stonestates[-1].copy()
    
    elif key == keyboard.Key.delete:
        stonestates = [Stone(num_facets)]
        cur_stone = stonestates[-1].copy()

    else:
        key = str(key)[1]
        cur_stone = stonestates[-1].copy()
        
        if key == "z": #suc 1
            cur_stone.tap1(1)
        
        if key == "h": #suc 2
            cur_stone.tap2(1)
        
        if key == "n": #suc 3
            cur_stone.tap3(1)

        
        if key == "u": #fail 1
            cur_stone.tap1(0)
        
        if key == "j": #fail 2
            cur_stone.tap2(0)
        
        if key == "m": #fail 3
            cur_stone.tap3(0)

        stonestates.append(cur_stone)


    clear_output(wait=True)
    print(cur_stone.string())
    arr = cur_stone.to_array()
    scores = np.array([get_score(arr, 0), get_score(arr, 1), get_score(arr, 2)])
    print(scores)
    print("Do row: ", np.argmax(scores) + 1)


num_facets = 9 # int(input("num facets: "))

stonestates = [Stone(num_facets)]


print(stonestates[-1].string())

listener = keyboard.Listener(on_press=on_press)
listener.start()
listener.join()

   *  *  *  *  *  *  *  *  * 9 Z U
                             0 H J
   _  _  _  _  _             0 N M
  Backspace: Undo, Delete: Reset, Esc: Exit
[-9.          9.13058066  8.39299607]
Do row:  2
Exiting
